In [47]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.model.CSVDFHandler import CSVDFHandler
from processing_pipeline.utilities.data_transformation import load_all_files
from utilities.csv.split_file_into_n_parts import split_file

In [48]:
from processing_pipeline.data_optimization.stage_cfg import S0NoiseFiltering

current_stage_cfg = S0NoiseFiltering
current_stage_cfg

StageConfig(name='s0', depends_on_fields=['sentence'], resulting_fields=['to_eliminate', 'reasoning'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s0_noise_filtering'), out_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s0_noise_filtering_o'), boolean_field_name='to_eliminate', next_stage=StageConfig(name='s1', depends_on_fields=['qa', 'sentence'], resulting_fields=['true_positive', 'reasoning'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s1_qa_relevance_check'), out_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s1_qa_relevance_check_o'), boolean_field_name='true_positive', next_stage=StageConfig(name='s2', depends_on_fields=['sentence'], resulting_fields=['related_to_arch', 'reasoning'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keyw

In [51]:
current_stage_cfg = current_stage_cfg.next_stage
current_stage_cfg

StageConfig(name='s3', depends_on_fields=['qa', 'sentence'], resulting_fields=['tactic', 'response'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction'), out_dir=None, boolean_field_name=None, next_stage=None, _apply_filters=<function <lambda> at 0x000001C0F9545580>, _on_load=<function StageConfig.<lambda>.<locals>.<lambda> at 0x000001C0F957F7E0>)

In [52]:
in_dir = current_stage_cfg.in_dir
out_path = AbsDirPath.SAMPLES / (current_stage_cfg.name + ".csv")
in_dir, out_path

(WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction'),
 WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/samples_2/s3.csv'))

In [53]:
df = load_all_files(in_dir)
df.shape

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\allenai.scispacy.v0.5.5.issue.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\google.deepvariant.v1.6.1.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\google.deepvariant.v1.6.1.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords_2\s3_tactic_extraction\googl

(217, 10)

In [54]:
current_stage_cfg.get_columns()

['s3_tactic', 's3_response']

In [55]:
for col in current_stage_cfg.get_columns():
    try:
        df = df[df[col].str.len() > 0]
    except:
        pass
df.shape

(217, 10)

In [56]:
sample_size = 384

In [57]:
# fake
sample_size = df.shape[0]

In [58]:
out_path.parent.mkdir(exist_ok=True)
df.sample(sample_size).to_csv(out_path)


In [12]:
n_parts = 5
split_file(out_path, n_parts, CSVDFHandler())